# Phase 1 

### Preprocessing

In [15]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

### Now load the pickle data into dataframes from the Output folder one directory above

In [16]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()
display(all_projects_bugreports.iloc[0])
display(all_projects_source_codes.iloc[2])
display(all_projects_bugreports.iloc[0].name)
display(all_projects_source_codes.iloc[2].name)

*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


fix                  [org.apache.commons.collections.map.flat3map.j...
text                                                               NaN
fixdate                                            2006-07-18 22:02:11
summary              Flat3Map.Entry.setValue() overwrites other Ent...
description          Flat3Map&amp;apos;s Entry objects will overwri...
project                                                    COLLECTIONS
average_precision                                                    0
Name: 217, dtype: object

filename            \gitrepo\src\java\org\apache\commons\collectio...
unprocessed_code    /*\n *  Licensed to the Apache Software Founda...
project                                                   COLLECTIONS
Name: 2, dtype: object

'217'

2

In [17]:
# need to get the file names in the bug report an source code to be the same

# there is no index connection between bugreports index (.name) and the index of the source code
display(all_projects_bugreports.iloc[0].fix)
display(all_projects_source_codes.iloc[217].filename)


array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

'\\gitrepo\\src\\java\\org\\apache\\commons\\collections\\list\\transformedlist.java'

## Removing composite varaibles

In [18]:
import re
#remove next line characters:
def remove_new_lines(text):
#     return str(x).replace('\n', '').replace('*', '').replace('/', '').replace('\\','').replace('\t','')
    text = str(text)
    COMBINE_WHITE_SPACE = re.compile(r"(?a:\s+)")
    text = COMBINE_WHITE_SPACE.sub(' ', text)
    return text.replace('*', '').replace('/', '').replace('\\','')
    
# clean up the various white space and remove some *
def clean_new_lines_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

# clean up the description and summary, they will both be used for the query
def clean_new_lines_bug_report(df):
    df.summary = df.summary.apply(remove_new_lines)
    df['description'] = df['description'].astype('|S')
    df.description = df.description.apply(remove_new_lines)
    return df

# need to reformat the source code so it can be compared to the bug reports fix array
def format_sc_filename(x):
  
    x = x.split('apache')
    if len(x) != 2:
        x = x[0].split('springframework')
        if len(x) != 2:
            x = x[0].split('wildfly')
            if len(x) == 2:
                x = 'org.wildfly' + x[1]
        else:
             x = 'org.springframework' + x[1]
    else:
         x = 'org.apache' + x[1]
    if len(x) == 1:
        x = x[0]
   
    x = str(x).replace("\\",".")
    return x

# apply the fixes to the filename 
def clean_sc_filepath(df):
    df.filename = df.filename.apply(format_sc_filename)
    return df



In [19]:
#splitting composite words
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_composite_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

def clean_composite_bug_report(df):
    df.summary = df.summary.apply(findCompositeWords)
    df.description = df.description.apply(findCompositeWords)
    return df


In [20]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# add the Java key words to the stop words
java_keywords = ["abstract", "assert**","assert", "boolean", "break", "byte", "case", "catch", "char", "const", "continue", "default", "do", "double", "else", "enum", "enum****" "extends", "final", "finally", "for", "goto","goto*", "if", "implements", "import", "instanceof", "int","interface", "long", "native", "new", "package", "private", "protected", "public", "return", "short", "static", "strictfp**","strictfp", "super", "switch", "synchornized", "this", "throw", "throws", "transient", "try", "void", "volatile", "while"]
java_operators = ["+", "-", "*", "/", "%", "+=", "-=", "*=", "/=", "++", "--", "==", "!=", "<", ">", "<=", ">=", ".", "[", "]", "(",")", "!", "~","instanceof", "<<", ">>", ">>>", "&", "^", "|", "&&", "||", "?", ":", "^=", "%=", "<<=", ">>=", ">>>=", "&="]
stop = java_keywords + java_operators
STOP_WORDS = ENGLISH_STOP_WORDS.union(stop)
# STOP_WORDS


### Start working on TF-IDF and Cosine similarity calculations

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clean up the unprocessed code column
def clean_source_df(df):
    df = clean_new_lines_source_code(df)
    df = clean_composite_source_code(df)
    df = clean_sc_filepath(df)
    return df

# add the summary and description together and clean the data
def clean_combine_bug_df(df):
    df = clean_new_lines_bug_report(df)
    df = clean_composite_bug_report(df)
    df["query"] = df["summary"] + df["description"]
    return df

# get clean versions of the dataframes
sc_df = clean_source_df(all_projects_source_codes)
br_df = clean_combine_bug_df(all_projects_bugreports)


# get a list of the projects 
print(sc_df.project.unique())
projects = sc_df.project.unique()

# group the data frames
# TODO ignore index here? Then maintain original index and concat the data frames
sc_grouped_df = sc_df.groupby(sc_df.project)
bg_grouped_df = br_df.groupby(br_df.project)

# example of getting a data frame for a single project
col_df = sc_grouped_df.get_group("COLLECTIONS")
display(col_df)




['COLLECTIONS' 'CONFIGURATION' 'IO' 'LANG' 'DATACMNS' 'DATAMONGO'
 'DATAREST' 'LDAP' 'SEC' 'SOCIALFB' 'SPR' 'ELY']


,filename,unprocessed_code,project
0,org.apache.commons.collections.arraystack.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,org.apache.commons.collections.bag.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,org.apache.commons.collections.bagutils.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,org.apache.commons.collections.beanmap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,org.apache.commons.collections.bidimap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
471,org.apache.commons.collections.set.testtransfo...,Licensed to the Apache Software Foundation ...,COLLECTIONS
472,org.apache.commons.collections.set.testtypedse...,Licensed to the Apache Software Foundation ...,COLLECTIONS
473,org.apache.commons.collections.set.testtypedso...,Licensed to the Apache Software Foundation ...,COLLECTIONS
474,org.apache.commons.collections.set.testunmodif...,Licensed to the Apache Software Foundation ...,COLLECTIONS


In [27]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_similarity(src_vect, query):
    cosSim = cosine_similarity(query, src_vect).flatten()
    return cosSim

# fit the vectorizer and transform data
def get_similarity(vect, src_vect, query_data):
    query = vect.transform(query_data)
    similarity = calculate_similarity(src_vect,query)
     
    return similarity

# remove the stem and stop words
# takes in an array of strings returns an array of strings
def stem_stop(text):
    stemmer = PorterStemmer()   #"english"
    text = text.split()
    text = [w for w in text if not w in STOP_WORDS]
    text = list(map(lambda x: stemmer.stem(x), text))
    text = ' '.join(text)
    text = text.strip()
    return text

# take in the source code df for a project and a signle query return scores
def generate_scores_list(source, query):
    scores_list = [] 
    # get a list of strings from the data frames to be vectorized
    source['unprocessed_code'] = source['unprocessed_code'].map(stem_stop)
    query["query"] = query["query"].map(stem_stop)
    query_str = query["query"].iloc[0]
     
    # fit a vectorizer to the data
    vect = TfidfVectorizer(min_df=1).fit(source['unprocessed_code'])
    src_vect = vect.transform(source['unprocessed_code'])
    display(src_vect)
    
    #get the similarity scores for the bug reports    
    #iterate through the query
    for q in query["query"]:
        similarity = get_similarity(vect, src_vect, [q])
        scores_list.append(similarity)
        # some sort of data structure (dict) to collect the queries to add it to the dataframe later
#         display(similarity)
    return scores_list
    
def generate_all_scores():
    
    all_bugs = []
    all_src = []
    # iterate through the list of 12 projects
    for proj in projects:
        # create dataframes for each project
        src_df = sc_grouped_df.get_group(proj)
        bug_df = bg_grouped_df.get_group(proj)
        
        # generate the scores list
        scores = generate_scores_list(src_df, bug_df)
    
        #append scores list to the bug dataframe
        bug_df["sim_vect"] = scores # the only way that the matrix is related to the src code 
                                    # is through the index.
        
        # Rank the source code files and know what file(s) the bug was in
        
        # CALCULATE THE MAP AND MRR HERE WITH A FUNCTION AND ADD IT TO THE BUGS DATAFRAME
    
        
        # maintain a list of all the dataframes
        all_bugs.append(bug_df)
        all_src.append(src_df)
    # concatenate all the data frames in order    
    all_bug_df = pd.concat(all_bugs, ignore_index=True)
    all_src_df = pd.concat(all_src, ignore_index=True)
    return all_bug_df, all_src_df

## TODO list
- Figure out index of the top 20 in the bug sim_vect
- Figure out index of files where the fix was ( this is in the bug report )
- Calculate MAP and MRR for each
- Generate graphs and report on the findings

In [26]:
bugs, sources = generate_all_scores()

C:\Users\kfrie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\kfrie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<476x3491 sparse matrix of type '<class 'numpy.float64'>'
	with 65214 stored elements in Compressed Sparse Row format>

array([0.05854813, 0.08970217, 0.00715676, 0.13971951, 0.2550643 ,
       0.0340014 , 0.00809668, 0.04066182, 0.09646529, 0.05696235,
       0.01317246, 0.01555954, 0.00981162, 0.01800695, 0.0348495 ,
       0.11432798, 0.02792165, 0.0707204 , 0.12558789, 0.40210781,
       0.16734536, 0.00964384, 0.19235734, 0.02816721, 0.02111962,
       0.04456904, 0.25135124, 0.23144774, 0.01133878, 0.05427212,
       0.31598891, 0.04868341, 0.14186802, 0.03296193, 0.17031487,
       0.28785933, 0.25832482, 0.27184969, 0.17324329, 0.14786105,
       0.01431774, 0.18082757, 0.08923396, 0.08029358, 0.02559319,
       0.03421888, 0.23396474, 0.296183  , 0.01018925, 0.00920178,
       0.29017429, 0.19961487, 0.02967568, 0.15316747, 0.21070003,
       0.00778536, 0.05498255, 0.05755438, 0.06996331, 0.04097079,
       0.01322058, 0.03808486, 0.22498182, 0.00874511, 0.07430436,
       0.02683223, 0.01059502, 0.04849728, 0.00534352, 0.04284966,
       0.00808682, 0.07999171, 0.00839534, 0.00801984, 0.06661

array([1.33887956e-02, 2.84006831e-02, 9.77221824e-03, 1.03451366e-01,
       1.66837922e-02, 3.97264016e-03, 6.26727601e-03, 3.87927071e-03,
       6.42863707e-03, 3.50451738e-03, 3.06022711e-03, 4.54764502e-03,
       1.08769484e-02, 1.51586127e-02, 5.92163051e-04, 1.50970118e-02,
       9.05339344e-03, 2.34591256e-03, 2.51130484e-02, 9.06691991e-03,
       5.47180801e-03, 1.17601458e-02, 8.80717932e-02, 2.11561336e-02,
       7.31662367e-04, 4.60259831e-03, 7.04651079e-03, 6.25483594e-03,
       8.33917422e-04, 9.99834582e-03, 2.52454295e-03, 3.39296823e-03,
       1.96072926e-03, 1.30595582e-02, 3.65127375e-02, 1.70773934e-02,
       1.59213572e-02, 2.33815834e-03, 1.44316508e-03, 2.18628716e-03,
       5.85747893e-03, 1.80173724e-02, 5.19457523e-03, 1.63251402e-02,
       2.92484448e-03, 1.04109227e-03, 2.19510516e-03, 3.54409661e-02,
       7.92466041e-03, 7.15666359e-03, 6.13972571e-03, 2.10157649e-02,
       4.70886740e-03, 2.24662274e-03, 4.12957698e-02, 5.12107095e-04,
      

array([4.85185084e-02, 1.65731022e-01, 5.23209007e-03, 2.34677685e-02,
       2.43802630e-02, 1.39398571e-02, 1.67973668e-01, 5.04585953e-02,
       1.24288029e-02, 1.32983161e-01, 4.90198740e-03, 6.01527619e-03,
       2.06640988e-02, 3.14242214e-02, 2.54725261e-02, 2.80460762e-01,
       1.10702878e-02, 9.08781225e-02, 1.02818172e-01, 1.09369065e-02,
       2.71395826e-02, 8.17037526e-02, 2.20368536e-02, 3.92015184e-02,
       1.87092428e-02, 2.03139754e-01, 6.49118612e-02, 5.40215759e-02,
       3.19954126e-03, 7.88915356e-02, 6.67854376e-03, 8.75629919e-02,
       1.16679881e-02, 2.62855377e-01, 6.99066320e-02, 6.20090854e-02,
       1.81863888e-02, 1.48831646e-01, 1.08152153e-01, 3.52309406e-02,
       6.45946434e-02, 7.08363315e-03, 1.37419717e-02, 3.38424650e-02,
       2.94838239e-02, 4.59015224e-02, 5.58290886e-02, 2.96278070e-02,
       2.09642665e-02, 8.16134077e-02, 7.01960077e-02, 2.83714737e-02,
       1.24570576e-02, 3.29938854e-02, 5.21258631e-02, 1.41250605e-02,
      

...

array([0.11097674, 0.00712102, 0.02228837, 0.05677428, 0.01620366,
       0.11147015, 0.00892802, 0.08184002, 0.05747997, 0.05897526,
       0.01343844, 0.12455642, 0.03529008, 0.02364022, 0.14950412,
       0.06207073, 0.03625791, 0.0547473 , 0.04668584, 0.03377529,
       0.12953727, 0.0280097 , 0.02991133, 0.03071299, 0.0644115 ,
       0.01176598, 0.06118101, 0.02344596, 0.0210314 , 0.07075255,
       0.06951962, 0.02155187, 0.02536565, 0.02229805, 0.04855653,
       0.01626656, 0.0186343 , 0.02094797, 0.02730642, 0.02426005,
       0.01219308, 0.05285537, 0.02099079, 0.04926515, 0.00805699,
       0.0018446 , 0.01119099, 0.00087419, 0.00121949, 0.00569712,
       0.02438928, 0.06823769, 0.00410382, 0.02999686, 0.02504649,
       0.02154839, 0.05312238, 0.03739942, 0.01846765, 0.00593982,
       0.02451845, 0.00386088, 0.00271287, 0.00264327, 0.01641871,
       0.00386143, 0.01785458, 0.02527145, 0.00218271, 0.02028201,
       0.01331508, 0.00906575, 0.01504067, 0.01715311, 0.00398

array([0.05376472, 0.01568693, 0.02081141, 0.00379744, 0.01809114,
       0.11678095, 0.02997333, 0.07075143, 0.03576776, 0.03892569,
       0.02585815, 0.02522475, 0.02808644, 0.01990453, 0.10018181,
       0.0317112 , 0.03189486, 0.03334765, 0.04262535, 0.02908715,
       0.02671032, 0.02557356, 0.04050174, 0.02475709, 0.03290707,
       0.02481429, 0.05110984, 0.03913646, 0.08595433, 0.07283818,
       0.0352544 , 0.03272792, 0.04604534, 0.04820627, 0.10078932,
       0.09127448, 0.05659804, 0.04946182, 0.08227947, 0.03417878,
       0.06920857, 0.06287401, 0.00361397, 0.07112105, 0.03246506,
       0.0137092 , 0.04101623, 0.00159932, 0.0008198 , 0.0276579 ,
       0.03818483, 0.04107058, 0.06862885, 0.08021884, 0.03264577,
       0.01252849, 0.08210138, 0.07916937, 0.06413279, 0.03733486,
       0.00759327, 0.00396043, 0.01855571, 0.03810945, 0.02653532,
       0.003961  , 0.01012917, 0.06258564, 0.03549855, 0.01514098,
       0.03730307, 0.03180748, 0.02901022, 0.05017352, 0.03323

array([0.28910818, 0.08488692, 0.07373466, 0.06125581, 0.04999213,
       0.28844068, 0.09444428, 0.2374422 , 0.03184792, 0.21256745,
       0.13678642, 0.14832455, 0.15317116, 0.11622372, 0.2329221 ,
       0.16313486, 0.15674064, 0.20367037, 0.12987078, 0.14952736,
       0.14865314, 0.13486285, 0.20314451, 0.07892383, 0.16928688,
       0.06230751, 0.15478427, 0.15786683, 0.21157122, 0.22378238,
       0.09850923, 0.08513882, 0.17631492, 0.34133303, 0.64905791,
       0.51856453, 0.21339138, 0.41285482, 0.44460513, 0.0745873 ,
       0.35643237, 0.45979679, 0.03799045, 0.34279131, 0.23634007,
       0.05917713, 0.17530907, 0.04543919, 0.01494682, 0.06902296,
       0.13182368, 0.19863159, 0.17195358, 0.262459  , 0.14330974,
       0.22687658, 0.36437493, 0.29746153, 0.31733028, 0.1227656 ,
       0.03039118, 0.03224033, 0.09695487, 0.1811429 , 0.13170813,
       0.02942347, 0.04752373, 0.25544411, 0.12789957, 0.04656079,
       0.05617357, 0.23563577, 0.23618218, 0.33800725, 0.14330

C:\Users\kfrie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 